In [2]:
import torchaudio
import math
import sys
sys.path.append("/home/irene/Documents/audio-diffusion-pytorch-trainer")

from main.separation import ContextualSeparator
from pathlib import Path
from audio_diffusion_pytorch import KarrasSchedule
import torch
from script.misc import hparams
import main.module_base
from main.dataset import PitchShift
from evaluation.evaluate_separation import si_snr
from IPython.display import Audio
device = torch.device("cuda")
ROOT_PATH = Path(".")
%load_ext autoreload
%autoreload 2

sampling_rate = 22050

# @markdown Generation length in seconds (will be rounded to be a power of 2 of sample_rate*length)
length = 10
length_samples = 2**math.ceil(math.log2(length * sampling_rate))

# @markdown Number of samples to generate
num_samples = 1

# @markdown Number of diffusion steps (higher tends to be better but takes longer to generate)
num_steps = 150
num_stems = 4

s1, sr1 = torchaudio.load('/home/irene/Documents/audio-diffusion-pytorch-trainer/data/Slack/test/bass/Track01881.wav')
s2, sr2 = torchaudio.load('/home/irene/Documents/audio-diffusion-pytorch-trainer/data/Slack/test/drums/Track01881.wav')
s3, sr3 = torchaudio.load('/home/irene/Documents/audio-diffusion-pytorch-trainer/data/Slack/test/guitar/Track01881.wav')
s4, sr4 = torchaudio.load('/home/irene/Documents/audio-diffusion-pytorch-trainer/data/Slack/test/piano/Track01881.wav')

s1 = torchaudio.functional.resample(s1, orig_freq=sr1, new_freq=sampling_rate)
s2 = torchaudio.functional.resample(s2, orig_freq=sr2, new_freq=sampling_rate)
s3 = torchaudio.functional.resample(s3, orig_freq=sr3, new_freq=sampling_rate)
s4 = torchaudio.functional.resample(s4, orig_freq=sr4, new_freq=sampling_rate)

#display(Audio(s1, rate = sampling_rate))
#display(Audio(s2, rate = sampling_rate))

start_sample = 100 * sampling_rate
s1 = s1.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
s2 = s2.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
s3 = s3.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
s4 = s4.reshape(1, 1, -1)[:, :, start_sample:start_sample + length_samples]
m = s1+s2+s3+s4

smin = 1e-4
smax = 1.0
rho = 7.0
sigma_schedule=KarrasSchedule(sigma_min=smin, sigma_max=smax, rho=rho)

s1_shifted = torchaudio.functional.pitch_shift(
                waveform=s1,
                sample_rate=sampling_rate,
                n_steps=1,
            )

def load_model(path):
  model = main.module_base.Model(**{**hparams, "in_channels": 4})
  model.load_state_dict(torch.load(path)["state_dict"])
  model.to(device);
  return model

model = load_model("/home/irene/Documents/audio-diffusion-pytorch-trainer/logs/ckpts/avid-darkness-164_epoch=419-valid_loss=0.015.ckpt")
separator = ContextualSeparator(model=model, stems=["bass", "drums", "guitar", "piano"], sigma_schedule=sigma_schedule)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
separations_hint = separator.separate_with_hint(
    mixture=m,
    source_with_hint=inpaint,
    mask=inpaint_mask,
    num_steps=num_steps,
)

#gen_mix = separations["bass"] + separations["drums"] + separations["guitar"] + separations["piano"]
print(si_snr(separations_hint["bass"], s1))
print(si_snr(separations_hint["drums"], s2))
print(si_snr(separations_hint["guitar"], s3))
print(si_snr(separations_hint["piano"], s4))

-29.84492301940918
15.887845993041992
0.6800469756126404
7.957810401916504


In [22]:
separations = separator.separate(
    mixture=m,
    num_steps=num_steps
)
#gen_mix = separations["bass"] + separations["drums"] + separations["guitar"] + separations["piano"]
print(si_snr(separations["bass"].unsqueeze(0), s1))
print(si_snr(separations["drums"].unsqueeze(0), s2))
print(si_snr(separations["guitar"].unsqueeze(0), s3))
print(si_snr(separations["piano"].unsqueeze(0), s4))
#display(Audio(separations["bass"].squeeze().squeeze().cpu().detach(), rate = sampling_rate))

In [4]:
gibbs_step = 10
sources = torch.cat([s1, s2, s3, s4], axis=1).to(device)
def generate_mask_and_sources(fixed_sources_idx=[]):
    mobile_sources_idx = list(set([0,1,2,3]) - set(fixed_sources_idx))
    mobile_sources_idx.sort()
    inpaint                = torch.randn(num_samples, num_stems, length_samples).to(device)
    inpaint[:, fixed_sources_idx, :]       = sources[:, fixed_sources_idx, :]
    inpaint_mask           = torch.ones_like(inpaint)
    inpaint_mask[:, mobile_sources_idx, :] = 0.
    return inpaint, inpaint_mask

In [28]:
inpaint, inpaint_mask = generate_mask_and_sources()

p_min = (1 - 0.95) * (10 - 1) / (10 - 0.95*10 + 0.95)
p_max = p_min / (1 - 0.95)
def schedule_prob(t, T, alpha=0.95, p_min=p_min, p_max=p_max):
    p_t = max(p_min, p_max - (t - 1) * (p_max - p_min) / (alpha * (T - 1)))
    return p_t

display(schedule_prob(0, 10.))

6.896551724137931

In [ ]:
for i in range(gibbs_steps):
    separations_hint = separator.separate_with_hint(
        mixture=m,
        source_with_hint=inpaint,
        mask=inpaint_mask,
        num_steps=num_steps,
    )
    p = schedule_prob(i, gibbs_steps, )
    fixed_sources_idx = tor
    inpaint, inpaint_mask = generate_mask_and_sources()
    